#Atividade PCM - Gustavo Alves

In [ ]:
# Dados iniciais

import numpy as np
import matplotlib.pyplot as plt



#Taxa de aprendizagem
n = 0.1
precisao = 1e-06
beta = 0.5
a = 0.9

#Pesos utilizados no treinamento antes do teste final, sem validação

# peso1 = np.random.rand(5,5)
# peso2 = np.random.rand(2,6)


In [ ]:

#Função de ativação

def sigmoid(u,beta):
    return 1 / (1 + np.exp(-beta*u))



In [ ]:
#derivaçao
# g′(u) = β. g(u). (1 − g(u))

def derivadaSigmoid(u,beta):
    return beta*sigmoid(u,beta)*(1 - sigmoid(u,beta))

In [ ]:
def saidaFinal(u):
  return np.around(u) #arredondar o valor de u  para 0 ou 1

In [ ]:
def calculaI(X, W):
    i = []
    for z in W:
      m = np.multiply(X, z)
      u = np.sum(m)
      i.append(u)
    return i

In [ ]:
def calculaY(i, beta):
    y= []
    for z in i:
      y.append(sigmoid(z,beta))
    return y


In [ ]:

def gradiente(d, Y,u,beta):
  arrayGradiente = [] 
  for i in range(len(d)):
    arrayGradiente.append((d[i]-Y[i])*derivadaSigmoid(u[i],beta))
  return arrayGradiente

In [ ]:
def gradienteCamadasEscondidas(u, arrayGradiente,W,beta):
    transpostadeW = np.array(W).T
    arrayGradiente2 = []
    for j in range(len(u)):
      result = np.multiply(arrayGradiente, transpostadeW[j])
      arrayGradiente2.append(np.sum(result)* derivadaSigmoid(u[j],beta))
    return  arrayGradiente2


In [ ]:
def ajustaPeso(peso,Y,arrayGradiente):
    for j in range(len(arrayGradiente)):
      for i in range(len(Y)):
        peso[j][i] = peso[j][i] + n*arrayGradiente[j]*Y[i] 
    return peso

In [ ]:
def ajustaPesoMomentum(peso,Y,arrayGradiente,pesoAntigo):
    pesoAnt= peso.copy()
    for j in range(len(arrayGradiente)):
      for i in range(len(Y)):
        peso[j][i] = peso[j][i] + n*arrayGradiente[j]*Y[i] + a* (peso[j][i] - pesoAntigo[j][i])
    return peso, pesoAnt


In [ ]:
def calculaErro(d, y):
  result =  1/2* np.sum(np.multiply(np.subtract(d, y), np.subtract(d, y))) 
  return result


In [ ]:
def separaArquivoValidacao(arquivo):
    for i in range(9):
      arquivo.readline()
    arrayTreinamento = []
    arraySetosa = []
    arrayVersicolor = []
    arrayVirginica = []
    for linha in arquivo:
      arrayLinha = linha.split(',')
      arrayLinha = [-1] + arrayLinha
      if arrayLinha[5].strip() == 'Iris-setosa':
        arrayLinha[5] = [0,0]
        arrayLinha[0:5] =np.array([float(i) for i in arrayLinha[0:5]])
        if len(arraySetosa) < 5:
          arraySetosa.append(arrayLinha)
        else:
          arrayTreinamento.append(arrayLinha)
      elif arrayLinha[5].strip() == 'Iris-versicolor':
        arrayLinha[5] = [0,1]
        arrayLinha[0:5] =np.array([float(i) for i in arrayLinha[0:5]])
        if len(arrayVersicolor) < 5:
          arrayVersicolor.append(arrayLinha)
        else:
          arrayTreinamento.append(arrayLinha)
      elif arrayLinha[5].strip() == 'Iris-virginica':
        arrayLinha[5] = [1, 0]
        arrayLinha[0:5] =np.array([float(i) for i in arrayLinha[0:5]])
        if len( arrayVirginica) < 5:
           arrayVirginica.append(arrayLinha)
        else:
          arrayTreinamento.append(arrayLinha)
    return arrayTreinamento, arraySetosa + arrayVersicolor + arrayVirginica

In [ ]:
def treinamento(dado, peso1, peso2, beta, precisao):
  eqmAnterior = float('inf');
  eqmAtual = 1;
  epoca= 0
  contador = 0
  while (abs(eqmAtual - eqmAnterior) > precisao):
    epoca = epoca +1
    eqmAnterior = eqmAtual
    for linha in dado[0]:
      contador = contador + 1;
      X = linha[0:5]
      d = linha[5]
      i1 = calculaI(X, peso1)
      y1 = [-1] + calculaY(i1, beta)
      i2 = calculaI(y1, peso2)
      y2 = calculaY(i2, beta)
      delta1 = gradiente(d, y2, i2,beta)
      peso2 = ajustaPeso(peso2,y1, delta1)
      delta2 = gradienteCamadasEscondidas(i1, delta1, peso2,beta)
      peso1 = ajustaPeso(peso1,X, delta2)
      eqmAtual = eqmAtual + calculaErro(d, y2)
    eqmAtual = eqmAtual/contador
  return epoca, abs(eqmAtual - eqmAnterior)

 


In [ ]:

def treinamentoMomentum(dado, peso1, peso2, beta, precisao):
  eqmAnterior = float('inf');
  eqmAtual = 1;
  epoca= 0
  contador = 0
  while (abs(eqmAtual - eqmAnterior) > precisao):
    epoca = epoca +1
    eqmAnterior = eqmAtual
    pesoAnterior1 = peso1
    pesoAnterior2= peso2
    for linha in dado[0]:
      contador = contador + 1;
      X = linha[0:5]
      d = linha[5]
      i1 = calculaI(X, peso1)
      y1 = [-1] + calculaY(i1, beta)
      i2 = calculaI(y1, peso2)
      y2 = calculaY(i2, beta)
      delta1 = gradiente(d, y2, i2,beta)
      peso2,pesoAnterior2= ajustaPesoMomentum(peso2,y1, delta1,pesoAnterior2)
      delta2 = gradienteCamadasEscondidas(i1, delta1, peso2,beta)
      peso1,pesoAnterior1 = ajustaPesoMomentum(peso1,X, delta2,pesoAnterior1)
      eqmAtual = eqmAtual + calculaErro(d, y2)
    eqmAtual = eqmAtual/contador
  return epoca, abs(eqmAtual - eqmAnterior)

 



In [ ]:

def teste(dado):
  acuracia = 0
  contador= 0
  for linha in dado[1]:
    contador = contador + 1
    X = linha[0:5]
    d = linha[5]
    i1 = calculaI(X, peso1)
    y1 = [-1] + calculaY(i1,beta)
    i2 = calculaI(y1, peso2)
    y2 = calculaY(i2,beta)
    saida = saidaFinal(y2)
    if np.array_equiv(saida, d):# verifica se matrizes são equivalentes
      acuracia = acuracia +1
  return acuracia/contador


#Treinamento e Validação

In [ ]:

arrayDados = []
acuracias = []
erros = []
epocas = []
for i in range(1,10):
  arquivo = open('iris-10dobscv-'+str(i)+'tra.dat', 'r') #nome do arquivo muda de acordo com o SO utilizado, no windows é assim
  treinamentoo, validacao = separaArquivoValidacao(arquivo)
  arrayDados.append( [treinamentoo, validacao ])
arquivo.close()

for i in arrayDados:

#altera a topologia de acordo com qtd de neuronios na matriz de pesos ou adiciona mais camadas
#colocando mais matrizes de peso  e reseta a cada iteração no caso do treinamento com validação
  peso1 = np.random.rand(5,5)
  peso2 = np.random.rand(2,6)

  epoca, erro = treinamento(i,peso1,peso2,beta,precisao) # treinamento sem fator de momentum
  # epoca, erro = treinamentoMomentum(i,peso1,peso2,beta,precisao) # para aplicar o fator de momentum
  acuracia = teste(i)
  acuracias.append(acuracia)
  epocas.append(epoca)
  erros.append(erro)
print('____Desvio Padrão_______')
print('Erros:',np.std(erros))
print( 'Epocas: ',np.std(epocas))
print('Acuracia:',np.std(acuracias))
print('\n')
print('___Médias_______________________')
print('Erros: ',np.average (erros))
print('Epocas: ', np.average (epocas))
print('Acuracia:', np.average(acuracias))
 




#Teste

In [ ]:
arrayTeste = []
acuracias = []
for i in range(1,10):
  arquivo = open('iris-10dobscv-'+str(i)+'tst.dat', 'r') #nome do arquivo muda de acordo com o SO utilizado, no windows é assim
  testee, validacao = separaArquivoValidacao(arquivo)
  arrayTeste.append( [testee, validacao ])
arquivo.close()
for i in arrayTeste:
  acuracia = teste(i)
  acuracias.append(acuracia)
print('___Acuracias______________________')
print('Desvio Padrão: ',np.std(acuracias))
print('Media: ',np.average(acuracias))


___Acuracias______________________
Desvio Padrão:  0.0
Media:  0.6666666666666666
